In [1]:
from urllib.parse import urlencode, unquote
import requests
import json
import xmltodict
import pandas as pd
import math
import numpy as np
from datetime import datetime, date, timedelta

In [2]:
url = "https://www.youthcenter.go.kr/opi/empList.do"

#### bizTycdSel
정책유형 : 코드값을 사용합니다. 여러개 선택 시 ,(comma)로 분리하여 전송합니다.
>
```
1. 취업지원 (004001) <=> 인력
- 004001001 : 교육훈련·체험·인턴
- 004001002 : 중소(중견)기업 취업지원
- 004001003 : 전문분야 취업지원
- 004001004 : 해외진출

2. 창업지원 (004002) <=> 창업
- 004002001 : R&D 지원
- 004002002 : 경영 지원
- 004002003 : 자본금 지원

3. 주거·금융 (004003) <=> 금융
- 004003001 : 생활비지원 및 금융 혜택
- 004003002 : 주거지원
- 004003003 : 학자금 지원

4. 생활·복지 (004004) <=> 기타
- 004004001 : 건강
- 004004002 : 문화

5. 정책참여 (004005) <=> 내수
- 004005001 : 정책제안
- 004005002 : 권리보호
- 004005003 : 지역발전

6. 코로나19 (004006) <=> 기타
- 004006001 : 기본소득지원
- 004006002 : 저소득층지원
- 004006003 : 재난피해지원
- 004006004 : 소득및일자리보전
- 004006005 : 기타 인센티브
- 004006006 : 심리지원
```
>
#### srchPolyBizSecd
- 지역코드 - 시,도 : 코드값을 사용합니다.
- 여러개 선택 시 ,(comma)로 분리하여 전송합니다.
- [코드 :시,도]
- 003002001 :서울 | 003002002 :부산 | 003002003 :대구 | 003002004 :인천 | 003002005 :광주 | 003002006 :대전 | 003002007 :울산 | 003002008 :경기 | - 003002009 :강원 | 003002010 :충북 | 003002011 :충남 | 003002012 :전북 | 003002013 :전남 | 003002014 :경북 | 003002015 :경남 | 003002016 :제주 | - 003002017 :세종 |

In [3]:
# queryString 입력 
queryString = "?" + urlencode(
    {
     "openApiVlak": "52ab18f81994346e24846b27",  # 마이페이지 > OpenAPI관리 에서 발급받은 인증키 (필수)
     "display" : "100",    # 출력건수, 기본값 10, 최대 100까지 가능합니다. (필수)
     "pageIndex": "1",      # 조회할 페이지, 기본값 1 (필수)
     #"srchPolicyId": "",  # 정책 ID : 상세화면 조회 시 필수
     #"query": "서울",       # 키워드 검색어
     "bizTycdSel": "004001002", # 중소(중견)기업 취업지원
     #"srchPolyBizSecd" : ""
    }
)

In [4]:
queryURL = url + queryString

In [5]:
queryURL

'https://www.youthcenter.go.kr/opi/empList.do?openApiVlak=52ab18f81994346e24846b27&display=100&pageIndex=1&bizTycdSel=004001002'

In [6]:
response = requests.get(queryURL)

In [8]:
jsonString = json.dumps(xmltodict.parse(response.text))

In [10]:
data = json.loads(jsonString)

In [12]:
data = data['empsInfo']['emp']

In [13]:
data = pd.DataFrame.from_dict(data)

In [14]:
data['plcyTpNm'] = data['plcyTpNm'].replace(['취업지원'],'인력')
data['plcyTpNm'] = data['plcyTpNm'].replace(['창업지원'],'창업')
data['plcyTpNm'] = data['plcyTpNm'].replace(['주거·금융'],'금융')
data['plcyTpNm'] = data['plcyTpNm'].replace(['생활·복지'],'기타')
data['plcyTpNm'] = data['plcyTpNm'].replace(['정책참여'],'내수')
data['plcyTpNm'] = data['plcyTpNm'].replace(['코로나19'],'기타')

#### srchPolyBizSecd
- 지역코드 - 시,도 : 코드값을 사용합니다.
- 여러개 선택 시 ,(comma)로 분리하여 전송합니다.
- [코드 :시,도]
- 003002001 :서울 | 003002002 :부산 | 003002003 :대구 | 003002004 :인천 | 003002005 :광주 | 003002006 :대전 | 003002007 :울산 | 003002008 :경기 | - 003002009 :강원 | 003002010 :충북 | 003002011 :충남 | 003002012 :전북 | 003002013 :전남 | 003002014 :경북 | 003002015 :경남 | 003002016 :제주 | - 003002017 :세종 |

In [ ]:
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002001(.*)','서울', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002002(.*)','부산', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002003(.*)','대구', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002004(.*)','인천', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002005(.*)','광주', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002006(.*)','대전', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002007(.*)','울산', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002008(.*)','경기', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002009(.*)','강원', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002010(.*)','충북', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002011(.*)','충남', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002012(.*)','전북', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002013(.*)','전남', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002014(.*)','경북', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002015(.*)','경남', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002016(.*)','제주', regex=True)
data['polyBizSecd'] = data['polyBizSecd'].str.replace('003002017(.*)','세종', regex=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rownum         100 non-null    object
 1   bizId          100 non-null    object
 2   polyBizSecd    100 non-null    object
 3   polyBizTy      100 non-null    object
 4   polyBizSjnm    100 non-null    object
 5   polyItcnCn     100 non-null    object
 6   plcyTpNm       100 non-null    object
 7   sporScvl       100 non-null    object
 8   sporCn         100 non-null    object
 9   ageInfo        100 non-null    object
 10  empmSttsCn     100 non-null    object
 11  accrRqisCn     100 non-null    object
 12  majrRqisCn     100 non-null    object
 13  splzRlmRqisCn  100 non-null    object
 14  cnsgNmor       100 non-null    object
 15  rqutPrdCn      100 non-null    object
 16  rqutProcCn     100 non-null    object
 17  jdgnPresCn     100 non-null    object
 18  rqutUrla       100 non-null    

In [16]:
data['plcyTpNm'].unique()

array(['인력'], dtype=object)

In [17]:
import sqlite3

In [20]:
con = sqlite3.connect("C:/ioneU/Data/Data_Processing/Data/IONEJOB.db")
data.to_sql('청년정책_v3',con)

100

In [21]:
con.commit()